In [1]:
# Set the output bucket to write to, dataproc service account must have write access
# Do not include trailing slash or "gs://"
output_bucket = "clingen-dataproc-workspace-kferrite"
# Set the TSV path to write into bucket. Can contain slash like "folder/file.tsv"
# Do not include leading slash
report_filename = "clinvar-annotation-af-revstatus.tsv"


In [2]:
import hail as hl
# `idempontent=True` is useful for running all cells in the notebook
hl.init(idempotent=True)

Running on Apache Spark version 2.4.5
SparkUI available at http://cg-hail-m.c.clingen-dev.internal:4040
Welcome to
     __  __     <>__
    / /_/ /__  __/ /
   / __  / _ `/ / /
  /_/ /_/\_,_/_/_/   version 0.2.57-582b2e31b8bd
LOGGING: writing to /home/hail/hail-20210202-1902-0.2.57-582b2e31b8bd.log


In [3]:
# utility functions for file placement
import subprocess
import os

def run_args(args, fail_on_stderr=False, success_codes=[0]) -> tuple: # (stdout,stderr,returncode)
    print(args)
    p = subprocess.Popen(args, stdout=subprocess.PIPE, stderr=subprocess.PIPE)
    stdout, stderr = p.communicate()
    if (fail_on_stderr and len(stderr) > 0) or (p.returncode not in success_codes):
        raise RuntimeError("command {} failed with code {}:{}".format(
            args, p.returncode, stderr))
    return (stdout, stderr, p.returncode)

def local_to_bucket(local_path:str, gcs_path:str):
    if not gcs_path.startswith("gs://"):
        gcs_path = "gs://{}/{}".format(output_bucket, gcs_path)
    args = ["gsutil", "cp", local_path, gcs_path]
    run_args(args)
    
def bucket_to_local(gcs_path:str, local_path:str):
    if not gcs_path.startswith("gs://"):
        gcs_path = "gs://{}/{}".format(output_bucket, gcs_path)
    args = ["gsutil", "cp", gcs_path, local_path]
    run_args(args)
    
def local_to_hdfs(local_path:str, hdfs_path:str):
    if not local_path.startswith("/"):
        local_path = os.path.join(os.getcwd(), local_path)
    args = ["hdfs", "dfs", "-rm", hdfs_path]
    run_args(args, success_codes=[0,1]) # Allow error
    args = ["hadoop", "fs", "-cp", "file://" + local_path, hdfs_path]
#     args = ["hdfs", "dfs", "-cp", "file://" + local_path, hdfs_path]
    run_args(args)
    
def hdfs_to_local(hdfs_path:str, local_path:str):
    if os.path.exists(local_path):
        os.remove(local_path)
    args = ["hdfs", "dfs", "-cp", hdfs_path, "file://" + local_path]
    run_args(args)

In [4]:
# Obtain desired thresholds
import io, re
import pandas as pd
import numpy as np

input_filename = "input_files/sheet3.tsv"
bucket_to_local(input_filename, input_filename)
# print(os.path.isfile(input_filename))
# local_to_hdfs(input_filename, input_filename)

pd_df = pd.read_csv(input_filename, sep='\t', dtype=str)
pd_df = pd_df.astype(object).replace(np.nan, None)
input_ds = hl.Table.from_pandas(pd_df)
input_ds.describe()
input_ds.show()
# with open(input_filename) as f_in:
#     variation_ids = [line.strip() for line in f_in if len(line) > 0]
print("Loaded {} variations".format(input_ds.count()))
# print(df)

['gsutil', 'cp', 'gs://clingen-dataproc-workspace-kferrite/input_files/sheet3.tsv', 'input_files/sheet3.tsv']
----------------------------------------
Global fields:
    None
----------------------------------------
Row fields:
    'Query': str 
    'CA ID': str 
    'ClinVar Name': str 
    'ClinVar ID': str 
    '37 Coorindates': str 
    '38 Coordinates': str 
    'Gene': str 
    'NM': str 
    'NP': str 
    'clinvar_variation_id': str 
----------------------------------------
Key: []
----------------------------------------


,,,,,,,,,
Query,CA ID,ClinVar Name,ClinVar ID,37 Coorindates,38 Coordinates,Gene,NM,NP,clinvar_variation_id
str,str,str,str,str,str,str,str,str,str
"""NM_007194.3:c.1100delC""","""CA288251""","""NM_007194.4(CHEK2):c.1100del (p.Thr367fs)""","""128042""","""NC_000022.10:g.29091857del""","""NC_000022.11:g.28695869del""","""CHEK2""","""NM_001005735.1:c.1229del""","""NP_001005735.1:p.Thr410MetfsTer15""","""128042"""
"""NM_007194.3:c.470T>C""","""CA117630""","""NM_007194.4(CHEK2):c.470T>C (p.Ile157Thr)""","""5591""","""NC_000022.10:g.29121087A>G""","""NC_000022.11:g.28725099A>G""","""CHEK2""","""NM_001005735.1:c.599T>C""","""NP_001005735.1:p.Ile200Thr""","""5591"""
"""NM_007194.3:c.1283C>T""","""CA250528""","""NM_007194.4(CHEK2):c.1283C>T (p.Ser428Phe)""","""5603""","""NC_000022.10:g.29091207G>A""","""NC_000022.11:g.28695219G>A""","""CHEK2""","""NM_001005735.1:c.1412C>T""","""NP_001005735.1:p.Ser471Phe""","""5603"""
"""NM_000130.4:c.1601G>A""","""CA114378""","""NM_000130.4(F5):c.1601G>A (p.Arg534Gln)""","""642""","""NC_000022.10:g.29091207G>A""","""NC_000001.11:g.169549811C>T""","""F5""","""NM_000130.4:c.1601G>A""","""NP_000121.2:p.Arg534Gln""","""642"""
"""NM_000506.4:c.*97G>A""","""CA325636""","""NM_000506.4(F2):c.*97G>A""","""13310""","""NC_000011.9:g.46761055G>A""","""NC_000011.10:g.46739505G>A""","""F2""","""NM_000506.3:c.*97G>A""","""NP_000497.1:p.=""","""13310"""
"""NM_000038.5:c.3920T>A""","""CA008761""","""NM_000038.6(APC):c.3920T>A (p.Ile1307Lys)""","""822""","""NC_000005.9:g.112175211T>A""","""NC_000005.10:g.112839514T>A""","""APC""","""NM_000038.5:c.3920T>A""","""NP_000029.2:p.Ile1307Lys""","""822"""
"""NM_005373.2:c.317C>T""","""CA806636""","""NM_005373.2(MPL):c.317C>T (p.Pro106Leu)""","""265248""","""NC_000001.10:g.43804317C>T""","""NC_000001.11:g.43338646C>T""","""MPL""","""NM_005373.2:c.317C>T""","""NP_005364.1:p.Pro106Leu""","""265248"""
"""NM_005373.2:c.117G>T""","""CA123787""","""NM_005373.2(MPL):c.117G>T (p.Lys39Asn)""","""14162""","""NC_000001.10:g.43803807G>T""","""NC_000001.11:g.43338136G>T""","""MPL""","""NM_005373.2:c.117G>T""","""NP_005364.1:p.Lys39Asn""","""14162"""


Loaded 71 variations


In [5]:
import io
import re

# Read gnomAD data as Hail Tables
ds_exomes = hl.read_table(
    "gs://gcp-public-data--gnomad/release/2.1.1/ht/exomes/gnomad.exomes.r2.1.1.sites.ht")
ds_exomes = ds_exomes.annotate(
    source="gnomAD Exomes")

ds_genomes = hl.read_table(
    "gs://gcp-public-data--gnomad/release/2.1.1/ht/genomes/gnomad.genomes.r2.1.1.sites.ht")
ds_genomes = ds_genomes.annotate(
    source="gnomAD Genomes")

# Can perform a union here if wanting both (ds = ds1.union(ds2))
def select_necessary_cols(ds):
    ds = ds.select(ds.freq, ds.faf, ds.vep, ds.source)
    return ds

gnomad_exomes = select_necessary_cols(ds_exomes)
gnomad_genomes = select_necessary_cols(ds_genomes)

gnomad = gnomad_genomes.union(gnomad_exomes, unify=True)


# Show the schema of the hail Table


In [6]:
gnomad.describe()
# gnomad.show()
# If exists in genomes and exomes, pick genome
# gnomad_locus_counts = (gnomad.group_by(gnomad.locus)
#                             .aggregate(count=hl.agg.count()))
# gnomad_locus_counts.show()

# gnomad = gnomad.annotate(
#     locus_count=gnomad.filter(gnomad.locus==gnomad.locus).count()
# )

gnomad.show()
# gnomad = gnomad.filter(
#     (gnomad.source == hl.str("gnomAD Exomes")) & (gnomad.filter(gnomad.locus==gnomad.locus).count() > 1)
#     #hl.any(lambda genome: genome.locus == gnomad.locus, gnomad_genomes)

# )



----------------------------------------
Global fields:
    'rf': struct {
        variants_by_type: dict<str, int32>, 
        feature_medians: dict<str, struct {
            variant_type: str, 
            n_alt_alleles: int32, 
            qd: float64, 
            pab_max: float64, 
            info_MQRankSum: float64, 
            info_SOR: float64, 
            info_InbreedingCoeff: float64, 
            info_ReadPosRankSum: float64, 
            info_FS: float64, 
            info_QD: float64, 
            info_MQ: float64, 
            info_DP: int32
        }>, 
        test_intervals: array<interval<locus<GRCh37>>>, 
        test_results: array<struct {
            rf_prediction: str, 
            rf_label: str, 
            n: int32
        }>, 
        features_importance: dict<str, float64>, 
        features: array<str>, 
        vqsr_training: bool, 
        no_transmitted_singletons: bool, 
        adj: bool, 
        rf_hash: str, 
        rf_snv_cutoff: struct {
     

+---------------+
| locus         |
+---------------+
| locus<GRCh37> |
+---------------+
| 1:10067       |
| 1:10108       |
| 1:10109       |
| 1:10114       |
| 1:10114       |
| 1:10119       |
| 1:10120       |
| 1:10128       |
| 1:10131       |
| 1:10132       |
+---------------+

+------------------------------------------------------------------------------+
| alleles                                                                      |
+------------------------------------------------------------------------------+
| array<str>                                                                   |
+------------------------------------------------------------------------------+
| ["T","TAACCCTAACCCTAACCCTAACCCTAACCCTAACCCTAACCC"]                           |
| ["CAACCCT","C"]                                                              |
| ["AACCCT","A"]                                                               |
| ["T","C"]                                                                    |
| ["TAACCCTAACCCTAACCCTAACCCTAACCCTAACCCCTAACCCTAACCCTAACCCTAACCCTAACCTAACC... |
| ["CT","C"]                                                                   |
| ["T","C"]                                                                    |
| ["ACCCTAACCCTAACCCTAAC","A"]                                                 |
| ["CT","C"]                                                                   |
| ["TAACCC","T"]                                                               |
+------------------------------------------------------------------------------+

+------------------------------------------------------------------------------+
| freq                                                                         |
+------------------------------------------------------------------------------+
| array<struct{AC: int32, AF: float64, AN: int32, homozygote_count: int32}>    |
+------------------------------------------------------------------------------+
| [(1,3.80e-05,26342,0),(3,9.55e-05,31414,0),(0,0.00e+00,13124,0),(1,3.75e-... |
| [(2,1.01e-03,1982,0),(2,6.49e-05,30832,0),(0,0.00e+00,1452,0),(1,2.27e-02... |
| [(19,6.42e-02,296,0),(31,1.01e-03,30766,0),(8,5.13e-02,156,0),(2,5.88e-02... |
| [(0,0.00e+00,8734,0),(8,2.56e-04,31192,0),(0,0.00e+00,4054,0),(0,0.00e+00... |
| [(1,1.14e-04,8734,0),(1,3.21e-05,31192,0),(0,0.00e+00,4054,0),(0,0.00e+00... |
| [(0,0.00e+00,24572,0),(4,1.28e-04,31190,0),(0,0.00e+00,11554,0),(0,0.00e+... |
| [(0,0.00e+00,12354,0),(9,2.89e-04,31190,0),(0,0.00e+00,5930,0),(0,0.00e+0... |
| [(1,3.95e-05,25286,0),(3,9.66e-05,31046,0),(1,8.18e-05,12218,0),(0,0.00e+... |
| [(1,3.66e-05,27296,0),(7,2.25e-04,31096,0),(0,0.00e+00,13358,0),(1,3.38e-... |
| [(0,0.00e+00,14196,0),(2,6.44e-05,31074,0),(0,0.00e+00,7030,0),(0,0.00e+0... |
+------------------------------------------------------------------------------+

+------------------------------------------------------------------------------+
| faf                                                                          |
+------------------------------------------------------------------------------+
| array<struct{meta: dict<str, str>, faf95: float64, faf99: float64}>          |
+------------------------------------------------------------------------------+
| [({"group":"adj"}},0.00e+00,0.00e+00),({"group":"adj"},"pop":"nfe"}},0.00... |
| [({"group":"adj"}},1.79e-04,1.79e-04),({"group":"adj"},"pop":"nfe"}},0.00... |
| [({"group":"adj"}},4.20e-02,4.20e-02),({"group":"adj"},"pop":"nfe"}},2.55... |
| [({"group":"adj"}},0.00e+00,0.00e+00),({"group":"adj"},"pop":"nfe"}},0.00... |
| [({"group":"adj"}},0.00e+00,0.00e+00),({"group":"adj"},"pop":"nfe"}},0.00... |
| [({"group":"adj"}},0.00e+00,0.00e+00),({"group":"adj"},"pop":"nfe"}},0.00... |
| [({"group":"adj"}},0.00e+00,0.00e+00),({"group":"adj"},"pop":"nfe"}},0.00... |
| [({"group":"adj"}},0.00e+00,0.00e+00),({"group":"adj"},"pop":"nfe"}},0.00... |
| [({"group":"adj"}},0.00e+00,0.00e+00),({"group":"adj"},"pop":"

In [7]:
"""
ds.freq has raw frequency information, including AN, AC, and pop label. This is an array of 
structs, at indices determined by the categories in ds.globals.freq_index_dict

ds.faf has filtered allele frequency information, including confidence intervals faf95 adn faf99.
This is an array of structs, at indices determined by the category map in ds.globals.faf_index_dict
"""

def add_popmax_af(ds):
    """
    Adds a popmax_faf and popmax_af_pop column to the ds Hail Table containing gnomAD fields.
    
    popmax_faf is a faf structure from the original ds, containing the maximum faf of the
    listed faf structures in the original ds, based on the filtering criteria 
    `default_faf_filter_type`. 
    
    The popmax_index_dict_key column contains the text field from the
    ds.globals.faf_index_dict which corresponds to each popmax_faf. This is similar to the
    ds.popmax_faf.meta["pop"] value but not exactly the same (gnomad_afr vs afr)
    
    Returns the updated ds.
    """
    # Identify indices in FAF field that correspond to the entire dataset (not a subset like non-cancer)
    # faf_index_map = [(k,v) for k, v in hl.eval(ds.globals.faf_index_dict).items() if k.startswith("gnomad_")]
    from enum import Enum
    class FafFilterType(Enum):
        # Each correponds to a filter func for a (k,v) of faf label to value
        GNOMAD_GLOBAL = lambda t: t[0] == "gnomad"
        GNOMAD_SUPERPOP = lambda t: t[0].startswith("gnomad_")
        ANY = lambda t: True

    # By default, filter to superpopulations aggregate faf
    default_faf_filter_type = FafFilterType.GNOMAD_SUPERPOP

    def faf_filter(faf_idx_tuple:tuple):
        return default_faf_filter_type(faf_idx_tuple)

    # Get list of the global faf_index_dict which meets the default_faf_filter criteria
    # This gives the indices of the desired populations, by default will take all top level populations
    faf_index_map = list(filter(faf_filter, [(k,v) for k,v in hl.eval(ds.globals.faf_index_dict).items()]))
    faf_indices = [v for k,v in faf_index_map]
    faf_labels = [k for k,v in faf_index_map]
    
    # Annotate table with popmax FAF
    
    # This only will return the maximum pop FAF for each
    # variant, even if multiple populations meet the criteria. 
    # If we want all matching populations, need an explode() call
    # to flatten the pop FAFs into a record per pop per variant
    
    ds = ds.annotate(
        # AF popmax meta.pop, faf95, faf99
        popmax_faf=hl.sorted(
            # Take only the FAF entries that correspond to the desired populations (faf_indices)
            hl.literal(faf_indices).map(lambda i: ds.faf[i]),
            # Sort by 95% confidence FAF
            lambda faf_entry: faf_entry.faf95,
            # Sort high to low
            reverse=True
        )[0] # Take the first entry with the highest FAF
        ,
        # Label of the freq_index_dict entry for this record's max pop
        popmax_index_dict_key=hl.sorted(
            # List of tuples of (poplabel, faf_index)
            list(zip(list(faf_labels), list(faf_indices))),

            # Take only the FAF entries that correspond to the entire dataset
            # Sort by 95% confidence FAF
            key=lambda tpl: ds.faf[tpl[1]].faf95,
            # Sort high to low
            reverse=True
        )[0][0] # Take the first entry, which has the highest FAF
    )
    
    ds = ds.annotate(
#         popmax_faf_pop_freq=ds.freq[ds.globals.freq_index_dict["gnomad_" + ds.popmax_faf.meta.get("pop")]]

        # ds.globals.freq_index_dict uses the same keys as ds.globals.faf_index_dict so
        # we can reuse ds.popmax_index_dict_key created above
        # Frequency info about the AF popmax population (AC, AF, AN, homozygote_count)
        popmax_faf_pop_freq=ds.freq[ds.globals.freq_index_dict[ds.popmax_index_dict_key]] 
    )
    
    return ds


gnomad_with_popmax = add_popmax_af(gnomad)
gnomad_with_popmax.show()

+---------------+
| locus         |
+---------------+
| locus<GRCh37> |
+---------------+
| 1:10067       |
| 1:10108       |
| 1:10109       |
| 1:10114       |
| 1:10114       |
| 1:10119       |
| 1:10120       |
| 1:10128       |
| 1:10131       |
| 1:10132       |
+---------------+

+------------------------------------------------------------------------------+
| alleles                                                                      |
+------------------------------------------------------------------------------+
| array<str>                                                                   |
+------------------------------------------------------------------------------+
| ["T","TAACCCTAACCCTAACCCTAACCCTAACCCTAACCCTAACCC"]                           |
| ["CAACCCT","C"]                                                              |
| ["AACCCT","A"]                                                               |
| ["T","C"]                                                                    |
| ["TAACCCTAACCCTAACCCTAACCCTAACCCTAACCCCTAACCCTAACCCTAACCCTAACCCTAACCTAACC... |
| ["CT","C"]                                                                   |
| ["T","C"]                                                                    |
| ["ACCCTAACCCTAACCCTAAC","A"]                                                 |
| ["CT","C"]                                                                   |
| ["TAACCC","T"]                                                               |
+------------------------------------------------------------------------------+

+------------------------------------------------------------------------------+
| freq                                                                         |
+------------------------------------------------------------------------------+
| array<struct{AC: int32, AF: float64, AN: int32, homozygote_count: int32}>    |
+------------------------------------------------------------------------------+
| [(1,3.80e-05,26342,0),(3,9.55e-05,31414,0),(0,0.00e+00,13124,0),(1,3.75e-... |
| [(2,1.01e-03,1982,0),(2,6.49e-05,30832,0),(0,0.00e+00,1452,0),(1,2.27e-02... |
| [(19,6.42e-02,296,0),(31,1.01e-03,30766,0),(8,5.13e-02,156,0),(2,5.88e-02... |
| [(0,0.00e+00,8734,0),(8,2.56e-04,31192,0),(0,0.00e+00,4054,0),(0,0.00e+00... |
| [(1,1.14e-04,8734,0),(1,3.21e-05,31192,0),(0,0.00e+00,4054,0),(0,0.00e+00... |
| [(0,0.00e+00,24572,0),(4,1.28e-04,31190,0),(0,0.00e+00,11554,0),(0,0.00e+... |
| [(0,0.00e+00,12354,0),(9,2.89e-04,31190,0),(0,0.00e+00,5930,0),(0,0.00e+0... |
| [(1,3.95e-05,25286,0),(3,9.66e-05,31046,0),(1,8.18e-05,12218,0),(0,0.00e+... |
| [(1,3.66e-05,27296,0),(7,2.25e-04,31096,0),(0,0.00e+00,13358,0),(1,3.38e-... |
| [(0,0.00e+00,14196,0),(2,6.44e-05,31074,0),(0,0.00e+00,7030,0),(0,0.00e+0... |
+------------------------------------------------------------------------------+

+------------------------------------------------------------------------------+
| faf                                                                          |
+------------------------------------------------------------------------------+
| array<struct{meta: dict<str, str>, faf95: float64, faf99: float64}>          |
+------------------------------------------------------------------------------+
| [({"group":"adj"}},0.00e+00,0.00e+00),({"group":"adj"},"pop":"nfe"}},0.00... |
| [({"group":"adj"}},1.79e-04,1.79e-04),({"group":"adj"},"pop":"nfe"}},0.00... |
| [({"group":"adj"}},4.20e-02,4.20e-02),({"group":"adj"},"pop":"nfe"}},2.55... |
| [({"group":"adj"}},0.00e+00,0.00e+00),({"group":"adj"},"pop":"nfe"}},0.00... |
| [({"group":"adj"}},0.00e+00,0.00e+00),({"group":"adj"},"pop":"nfe"}},0.00... |
| [({"group":"adj"}},0.00e+00,0.00e+00),({"group":"adj"},"pop":"nfe"}},0.00... |
| [({"group":"adj"}},0.00e+00,0.00e+00),({"group":"adj"},"pop":"nfe"}},0.00... |
| [({"group":"adj"}},0.00e+00,0.00e+00),({"group":"adj"},"pop":"nfe"}},0.00... |
| [({"group":"adj"}},0.00e+00,0.00e+00),({"group":"adj"},"pop":"

In [8]:
# Import ClinVar VCF as Hail Table
# clinvar = hl.import_vcf("/path/to/clinvar.vcf.gz", force_bgz=True, drop_samples=True, skip_invalid_loci=True).rows()

# Download clinvar BGZF
import os, requests, subprocess

# Function to download a file to a localpath. ClinVar VCF is small enough to download to dataproc default local disk.
def download_to_file(url, filepath):
    r = requests.get(url, stream=True)
    if r.status_code != 200:
        raise RuntimeError("Failed to obtain ClinVar VCF:{}\n{}".format(r.status_code))
    with open(filepath, "wb") as fout: 
        for chunk in r.iter_content(chunk_size=1024): 
             if chunk:
                 fout.write(chunk)
# This url always points to the latest dump file, updated periodically by ClinVar
clinvar_vcf_url = "https://ftp.ncbi.nlm.nih.gov/pub/clinvar/vcf_GRCh38/clinvar.vcf.gz"
clinvar_vcf_localpath = "/home/hail/clinvar.vcf.gz"
clinvar_vcf_hdfs = "clinvar.vcf.gz"
download_to_file(clinvar_vcf_url, clinvar_vcf_localpath)
assert(os.path.exists(clinvar_vcf_localpath))
print("Downloaded ClinVar VCF, file size (expecting ~30M): %d" % os.path.getsize(clinvar_vcf_localpath))

# Hail needs the file in HDFS
local_to_hdfs(clinvar_vcf_localpath, clinvar_vcf_hdfs)
# p = subprocess.Popen(["hdfs", "dfs", "-cp", "file://" + clinvar_vcf_localpath, clinvar_vcf_hdfs])
# print(p.communicate())

clinvar = hl.import_vcf(
    clinvar_vcf_hdfs,
    force_bgz=True,
    drop_samples=True, 
    skip_invalid_loci=True
).rows()
print("Imported {} records from ClinVar".format(clinvar.count()))


Downloaded ClinVar VCF, file size (expecting ~30M): 34706610
['hdfs', 'dfs', '-rm', 'clinvar.vcf.gz']
['hadoop', 'fs', '-cp', 'file:///home/hail/clinvar.vcf.gz', 'clinvar.vcf.gz']


2021-02-02 19:03:06 Hail: INFO: Coerced sorted dataset


Imported 785521 records from ClinVar


In [9]:
# clinvar.describe()
# clinvar.show()

In [10]:
# Filter clinvar to those in input set
input_variation_ids = input_ds.clinvar_variation_id.collect()
clinvar_filtered = clinvar.filter(
    # Return true if clinvar.rsid value matches any input variation id 
    hl.any(lambda rec: rec == clinvar.rsid, input_variation_ids)
)
print("ClinVar filtered to {} rows".format(clinvar_filtered.count()))

2021-02-02 19:03:13 Hail: INFO: Coerced sorted dataset


ClinVar filtered to 67 rows


In [11]:
# Filter gnomad to records in input_ds
input_clinvar = input_ds.annotate(
    clinvar=clinvar.key_by(clinvar.rsid)[input_ds.clinvar_variation_id]
)
input_clinvar = input_clinvar.key_by(
    input_clinvar.clinvar.locus, input_clinvar.clinvar.alleles
)
input_clinvar.describe()
# input_clinvar.show()

----------------------------------------
Global fields:
    None
----------------------------------------
Row fields:
    'Query': str 
    'CA ID': str 
    'ClinVar Name': str 
    'ClinVar ID': str 
    '37 Coorindates': str 
    '38 Coordinates': str 
    'Gene': str 
    'NM': str 
    'NP': str 
    'clinvar_variation_id': str 
    'clinvar': struct {
        locus: locus<GRCh37>, 
        alleles: array<str>, 
        qual: float64, 
        filters: set<str>, 
        info: struct {
            AF_ESP: float64, 
            AF_EXAC: float64, 
            AF_TGP: float64, 
            ALLELEID: int32, 
            CLNDN: array<str>, 
            CLNDNINCL: array<str>, 
            CLNDISDB: array<str>, 
            CLNDISDBINCL: array<str>, 
            CLNHGVS: array<str>, 
            CLNREVSTAT: array<str>, 
            CLNSIG: array<str>, 
            CLNSIGCONF: array<str>, 
            CLNSIGINCL: array<str>, 
            CLNVC: str, 
            CLNVCSO: str, 
           

In [12]:
# Filter gnomad to locuses in input_clinvar
icg = input_clinvar.annotate(
    gnomad=gnomad[input_clinvar.locus, input_clinvar.alleles]
)
icg.describe()
# icg.show()

----------------------------------------
Global fields:
    None
----------------------------------------
Row fields:
    'Query': str 
    'CA ID': str 
    'ClinVar Name': str 
    'ClinVar ID': str 
    '37 Coorindates': str 
    '38 Coordinates': str 
    'Gene': str 
    'NM': str 
    'NP': str 
    'clinvar_variation_id': str 
    'clinvar': struct {
        locus: locus<GRCh37>, 
        alleles: array<str>, 
        qual: float64, 
        filters: set<str>, 
        info: struct {
            AF_ESP: float64, 
            AF_EXAC: float64, 
            AF_TGP: float64, 
            ALLELEID: int32, 
            CLNDN: array<str>, 
            CLNDNINCL: array<str>, 
            CLNDISDB: array<str>, 
            CLNDISDBINCL: array<str>, 
            CLNHGVS: array<str>, 
            CLNREVSTAT: array<str>, 
            CLNSIG: array<str>, 
            CLNSIGCONF: array<str>, 
            CLNSIGINCL: array<str>, 
            CLNVC: str, 
            CLNVCSO: str, 
           

In [13]:
import time

# start = time.time()
# clinvar_gnomad = clinvar_filtered.annotate(
#     gnomad=gnomad[clinvar_filtered.locus, clinvar_filtered.alleles]
# )
# # clinvar_gnomad = clinvar_filtered.join(gnomad, how="left")
# print("Clinvar joined with gnomad with %d rows" % clinvar_gnomad.count())
# clinvar_gnomad.show()
# end = time.time()
# print("clinvar <- gnomad join and filter took %.6f sec" % (end-start))

In [14]:
cg_popmax = icg

# Add freq + faf global annotations from gnomad to new table
cg_popmax = cg_popmax.annotate_globals(freq_index_dict=hl.eval(gnomad.globals.freq_index_dict))
cg_popmax = cg_popmax.annotate_globals(faf_index_dict=hl.eval(gnomad.globals.faf_index_dict))
# Pull gnomad faf and freq to top level
cg_popmax = cg_popmax.annotate(faf=cg_popmax.gnomad.faf)
cg_popmax = cg_popmax.annotate(freq=cg_popmax.gnomad.freq)

cg_popmax.describe()

# Compute popmax and annotate
with_popmax = add_popmax_af(cg_popmax)


----------------------------------------
Global fields:
    'freq_index_dict': dict<str, int32> 
    'faf_index_dict': dict<str, int32> 
----------------------------------------
Row fields:
    'Query': str 
    'CA ID': str 
    'ClinVar Name': str 
    'ClinVar ID': str 
    '37 Coorindates': str 
    '38 Coordinates': str 
    'Gene': str 
    'NM': str 
    'NP': str 
    'clinvar_variation_id': str 
    'clinvar': struct {
        locus: locus<GRCh37>, 
        alleles: array<str>, 
        qual: float64, 
        filters: set<str>, 
        info: struct {
            AF_ESP: float64, 
            AF_EXAC: float64, 
            AF_TGP: float64, 
            ALLELEID: int32, 
            CLNDN: array<str>, 
            CLNDNINCL: array<str>, 
            CLNDISDB: array<str>, 
            CLNDISDBINCL: array<str>, 
            CLNHGVS: array<str>, 
            CLNREVSTAT: array<str>, 
            CLNSIG: array<str>, 
            CLNSIGCONF: array<str>, 
            CLNSIGINCL: arra

In [15]:
with_popmax.describe()

----------------------------------------
Global fields:
    'freq_index_dict': dict<str, int32> 
    'faf_index_dict': dict<str, int32> 
----------------------------------------
Row fields:
    'Query': str 
    'CA ID': str 
    'ClinVar Name': str 
    'ClinVar ID': str 
    '37 Coorindates': str 
    '38 Coordinates': str 
    'Gene': str 
    'NM': str 
    'NP': str 
    'clinvar_variation_id': str 
    'clinvar': struct {
        locus: locus<GRCh37>, 
        alleles: array<str>, 
        qual: float64, 
        filters: set<str>, 
        info: struct {
            AF_ESP: float64, 
            AF_EXAC: float64, 
            AF_TGP: float64, 
            ALLELEID: int32, 
            CLNDN: array<str>, 
            CLNDNINCL: array<str>, 
            CLNDISDB: array<str>, 
            CLNDISDBINCL: array<str>, 
            CLNHGVS: array<str>, 
            CLNREVSTAT: array<str>, 
            CLNSIG: array<str>, 
            CLNSIGCONF: array<str>, 
            CLNSIGINCL: arra

In [16]:
# filtered_ds = gn_cv_input
# filtered_ds = filtered_ds.filter(
#     ~hl.is_missing(filtered_ds.input)
# )
# filtered_ds = filtered_ds.filter(
#     ~hl.is_missing(filtered_ds.clinvar_variation_id)
# )

# filtered_ds.describe()

# # filtered_ds = gnomad_clinvar_ds.filter(
# #     hl.literal(input_ds.clinvar_variation_id.collect()).contains(clinvar.rsid)
# # )
# print("Filtered {} clinvar records to {} based on input".format(
#     filtered_ds.count(), filtered_ds.count()))

In [17]:
# Join clinvar+gnomad to input table
# input_by_varid = input_ds.key_by("clinvar_variation_id")
# cg_popmax_by_varid = cg_popmax.key_by(cg_popmax.rsid)

# input_by_varid = input_by_varid.annotate(
#     annotations=cg_popmax_by_varid[input_by_varid.clinvar_variation_id]
# )
# cg_popmax_by_varid.describe()

In [18]:
list(with_popmax.row.keys())

['Query',
 'CA ID',
 'ClinVar Name',
 'ClinVar ID',
 '37 Coorindates',
 '38 Coordinates',
 'Gene',
 'NM',
 'NP',
 'clinvar_variation_id',
 'clinvar',
 'locus',
 'alleles',
 'gnomad',
 'faf',
 'freq',
 'popmax_faf',
 'popmax_index_dict_key',
 'popmax_faf_pop_freq']

In [19]:
# with_popmax.key.collect()
input_table_columns = list(input_ds.row.keys())
print(input_table_columns)

['Query', 'CA ID', 'ClinVar Name', 'ClinVar ID', '37 Coorindates', '38 Coordinates', 'Gene', 'NM', 'NP', 'clinvar_variation_id']


In [20]:
output_ds = with_popmax
# Query	CA ID	ClinVar Name	ClinVar ID	37 Coorindates	38 Coordinates	Gene	NM	NP	clinvar_variation_id
output_ds = output_ds.annotate(
    # already keyed by clinvar_variation_id
#     clinvar_variation_id=output_ds.clinvar_variation_id,
    clinvar_review_status=hl.delimit(output_ds.clinvar.info["CLNREVSTAT"], ","),
    clinvar_significance=hl.delimit(output_ds.clinvar.info["CLNSIG"], ","),
    clinvar_significance_interpretations=hl.delimit(output_ds.clinvar.info["CLNSIGCONF"], ","),
    # Hail parses the CLNDN (and related like CLNDNINCL) incorrectly
    # Since ',' is allowed in condition names, ClinVar uses '|' to separate them
    # But Hail separates into an array based on ',' instead of '|'
    # If we re-join the string with ',' it will match that from ClinVar
    clinvar_conditions=hl.delimit(output_ds.clinvar.info["CLNDN"], ","),
    popmax_pop = output_ds.popmax_faf.meta["pop"],
    popmax_ac = output_ds.popmax_faf_pop_freq.AC,
    popmax_an = output_ds.popmax_faf_pop_freq.AN,
)
output_ds = output_ds.select(
    *(input_table_columns + 
      ['clinvar_review_status', 
      'clinvar_significance',
      'clinvar_significance_interpretations',
      'clinvar_conditions',
      'popmax_pop',
      'popmax_ac',
      'popmax_an'])
)
output_ds.describe()


In [21]:
# Select desired output fields (columns are ordered as provided)
# output_ds = with_popmax
# # Query	CA ID	ClinVar Name	ClinVar ID	37 Coorindates	38 Coordinates	Gene	NM	NP	clinvar_variation_id
# output_ds = output_ds.select(
#     # already keyed by clinvar_variation_id
#     clinvar_review_status=hl.delimit(output_ds.clinvar.info["CLNREVSTAT"], ","),
#     clinvar_significance=hl.delimit(output_ds.clinvar.info["CLNSIG"], ","),
#     clinvar_significance_interpretations=hl.delimit(output_ds.clinvar.info["CLNSIGCONF"], ","),
#     # Hail parses the CLNDN (and related like CLNDNINCL) incorrectly
#     # Since ',' is allowed in condition names, ClinVar uses '|' to separate them
#     # But Hail separates into an array based on ',' instead of '|'
#     # If we re-join the string with ',' it will match that from ClinVar
#     clinvar_conditions=hl.delimit(output_ds.clinvar.info["CLNDN"], ","),
    
#     # gnomad popmax things
#     popmax_pop = output_ds.popmax_faf.meta["pop"],
#     popmax_ac = output_ds.popmax_faf_pop_freq.AC,
#     popmax_an = output_ds.popmax_faf_pop_freq.AN,
# )

# output_ds = output_ds.order_by(
#     hl.int(output_ds.clinvar_variation_id) # Assume all clinvar variation ids are integers
# )

# output_ds.describe()

# Export to TSV
import time
print("Starting export to %s" % report_filename)
start_time = time.time()
output_ds.export(report_filename)
end_time = time.time()
print("Export took %.2f seconds" % (end_time - start_time))

Starting export to clinvar-annotation-af-revstatus.tsv


2021-02-02 19:03:19 Hail: INFO: Ordering unsorted dataset with network shuffle
2021-02-02 19:03:22 Hail: INFO: Coerced sorted dataset
2021-02-02 19:03:28 Hail: INFO: Ordering unsorted dataset with network shuffle
2021-02-02 19:03:39 Hail: INFO: Ordering unsorted dataset with network shuffle
2021-02-02 20:29:57 Hail: INFO: merging 4 files totalling 38.3K...


Export took 5198.81 seconds


2021-02-02 20:29:57 Hail: INFO: while writing:
    clinvar-annotation-af-revstatus.tsv
  merge time: 458.856ms


In [22]:
# The export is in HDFS now, copy to machine-local file
report_localpath = os.path.join(os.getcwd(), report_filename)
hdfs_to_local(report_filename, report_localpath)

['hdfs', 'dfs', '-cp', 'clinvar-annotation-af-revstatus.tsv', 'file:///home/hail/clinvar-annotation-af-revstatus.tsv']


In [23]:
# Upload to bucket and filepath set at top of notebook
print("Uploading {} bytes to GCS".format(os.path.getsize(report_localpath)))
local_to_bucket(report_localpath, report_filename)

Uploading 39198 bytes to GCS
['gsutil', 'cp', '/home/hail/clinvar-annotation-af-revstatus.tsv', 'gs://clingen-dataproc-workspace-kferrite/clinvar-annotation-af-revstatus.tsv']
